In [1]:
from utils import *
import numpy as np
import torch, os
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV

In [2]:
# def scaling_embeddings(embeddings):
#     for i in range(len(embeddings)):
#         embeddings[i] = StandardScaler().fit_transform(embeddings[i])
#     return embeddings

def reshape_embeddings(embeddings):
    if embeddings.shape[1] == 1:
        embeddings = embeddings.squeeze(1)
    return embeddings

In [3]:
dynahate_labels_train = process_labels(read_labels("dynahate", "train"))
dynahate_labels_dev = process_labels(read_labels("dynahate", "dev"))
dynahate_labels_test = process_labels(read_labels("dynahate", "test"))

latenthatred_labels_train = read_labels("latenthatred", "train")
latenthatred_labels_dev = read_labels("latenthatred", "dev")
latenthatred_labels_test = read_labels("latenthatred", "test")

olid_labels_train = read_labels("olid", "train")
olid_labels_dev = read_labels("olid", "dev")
olid_labels_test = read_labels("olid", "test")

In [4]:
mlp = MLPClassifier()
gridsearch = GridSearchCV(
    mlp,
    param_grid={
        "hidden_layer_sizes": [(128), (128,64)],
        "activation": ["relu"],
        "solver": ["adam"],
        "learning_rate_init": [0.001, 0.0001],
        "learning_rate": ["adaptive"],
        "early_stopping": [True],
        "max_iter": [10000]
    },
    verbose=4,
    n_jobs=os.cpu_count()//3,
    cv=5,
)

In [5]:
dynahate_labels_train_dev = np.concatenate((dynahate_labels_train, dynahate_labels_dev))
latenthatred_labels_train_dev = np.concatenate((latenthatred_labels_train, latenthatred_labels_dev))
olid_labels_train_dev = np.concatenate((olid_labels_train, olid_labels_dev))

In [6]:
hatebert_dynahate_train_embeddings = reshape_embeddings(hatebert_dynahate_train_embeddings)
hatebert_dynahate_dev_embeddings = reshape_embeddings(hatebert_dynahate_dev_embeddings)
hatebert_dynahate_test_embeddings = reshape_embeddings(hatebert_dynahate_test_embeddings)

hatebert_latenthatred_train_embeddings = reshape_embeddings(hatebert_latenthatred_train_embeddings)
hatebert_latenthatred_dev_embeddings = reshape_embeddings(hatebert_latenthatred_dev_embeddings)
hatebert_latenthatred_test_embeddings = reshape_embeddings(hatebert_latenthatred_test_embeddings)

hatebert_olid_train_embeddings = reshape_embeddings(hatebert_olid_train_embeddings)
hatebert_olid_dev_embeddings = reshape_embeddings(hatebert_olid_dev_embeddings)
hatebert_olid_test_embeddings = reshape_embeddings(hatebert_olid_test_embeddings)

### HateBERT

#### DynaHate

In [7]:
hatebert_dynahate_train_dev = np.concatenate((hatebert_dynahate_train_embeddings, hatebert_dynahate_dev_embeddings))

grid_results = gridsearch.fit(hatebert_dynahate_train_dev, dynahate_labels_train_dev)

best_params = grid_results.best_params_
mlp = grid_results.best_estimator_

print("Best params: ", best_params)

train_dev_preds = mlp.predict(hatebert_dynahate_train_dev)
test_preds = mlp.predict(hatebert_dynahate_test_embeddings)

computeAllScores(train_dev_preds, test_preds, dynahate_labels_train_dev, dynahate_labels_test, "Results/hatebert_dynahate")
save_model(mlp, "Saves/hatebert_dynahate")

Fitting 5 folds for each of 4 candidates, totalling 20 fits
Best params:  {'activation': 'relu', 'early_stopping': True, 'hidden_layer_sizes': (128, 64), 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'max_iter': 10000, 'solver': 'adam'}
Accuracy Train Dev:  0.7236927398444253
Accuracy Test:  0.6895631067961165
Weighted F1 Train Dev:  0.7227959439185933
Weighted F1 Test:  0.6887506726581016
Macro F1 Train Dev:  0.7205699113268926
Macro F1 Test:  0.6848659301131108
Micro F1 Train Dev:  0.7236927398444254
Micro F1 Test:  0.6895631067961165
Weighted Recall Train Dev:  0.7236927398444253
Weighted Recall Test:  0.6895631067961165
Macro Recall Train Dev:  0.7198139485547681
Macro Recall Test:  0.6842114535599056
Micro Recall Train Dev:  0.7236927398444253
Micro Recall Test:  0.6895631067961165
Confusion Matrix Train Dev: 
[[11440  5677]
 [ 4553 15354]]
Confusion Matrix Test: 
[[1169  683]
 [ 596 1672]]


#### LatentHatred

In [8]:
hatebert_latenthatred_train_dev = np.concatenate((hatebert_latenthatred_train_embeddings, hatebert_latenthatred_dev_embeddings))

grid_results = gridsearch.fit(hatebert_latenthatred_train_dev, latenthatred_labels_train_dev)

best_params = grid_results.best_params_
mlp = grid_results.best_estimator_

print("Best params: ", best_params)

train_dev_preds = mlp.predict(hatebert_latenthatred_train_dev)
test_preds = mlp.predict(hatebert_latenthatred_test_embeddings)

computeAllScores(train_dev_preds, test_preds, latenthatred_labels_train_dev, latenthatred_labels_test, "Results/hatebert_latenthatred")
save_model(mlp, "Saves/hatebert_latenthatred")

Fitting 5 folds for each of 4 candidates, totalling 20 fits
Best params:  {'activation': 'relu', 'early_stopping': True, 'hidden_layer_sizes': (128, 64), 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'max_iter': 10000, 'solver': 'adam'}
Accuracy Train Dev:  0.6969584109248914
Accuracy Test:  0.6787709497206704
Weighted F1 Train Dev:  0.6753650038527814
Weighted F1 Test:  0.655091815784592
Macro F1 Train Dev:  0.4742929768472124
Macro F1 Test:  0.45001570345239505
Micro F1 Train Dev:  0.6969584109248914
Micro F1 Test:  0.6787709497206704
Weighted Recall Train Dev:  0.6969584109248914
Weighted Recall Test:  0.6787709497206704
Macro Recall Train Dev:  0.46906116894524996
Macro Recall Test:  0.4477083184835815
Micro Recall Train Dev:  0.6969584109248914
Micro Recall Test:  0.6787709497206704
Confusion Matrix Train Dev: 
[[8454 1474   17]
 [2585 2737   25]
 [ 341  440   37]]
Confusion Matrix Test: 
[[2814  526    6]
 [ 930  822    1]
 [ 116  146    9]]


#### OLID

In [9]:
hatebert_olid_train_dev = np.concatenate((hatebert_olid_train_embeddings, hatebert_olid_dev_embeddings))

grid_results = gridsearch.fit(hatebert_olid_train_dev, olid_labels_train_dev)

best_params = grid_results.best_params_
mlp = grid_results.best_estimator_

print("Best params: ", best_params)

train_dev_preds = mlp.predict(hatebert_olid_train_dev)
test_preds = mlp.predict(hatebert_olid_test_embeddings)

computeAllScores(train_dev_preds, test_preds, olid_labels_train_dev, olid_labels_test, "Results/hatebert_olid")

save_model(mlp, "Saves/hatebert_olid")

Fitting 5 folds for each of 4 candidates, totalling 20 fits
Best params:  {'activation': 'relu', 'early_stopping': True, 'hidden_layer_sizes': 128, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'max_iter': 10000, 'solver': 'adam'}
Accuracy Train Dev:  0.7484894259818731
Accuracy Test:  0.791860465116279
Weighted F1 Train Dev:  0.7359465513513125
Weighted F1 Test:  0.7691054391479656
Macro F1 Train Dev:  0.6914654304569916
Macro F1 Test:  0.6912053502474329
Micro F1 Train Dev:  0.748489425981873
Micro F1 Test:  0.791860465116279
Weighted Recall Train Dev:  0.7484894259818731
Weighted Recall Test:  0.791860465116279
Macro Recall Train Dev:  0.680892122583299
Macro Recall Test:  0.670497311827957
Micro Recall Train Dev:  0.7484894259818731
Micro Recall Test:  0.791860465116279
Confusion Matrix Train Dev: 
[[7801 1039]
 [2291 2109]]
Confusion Matrix Test: 
[[586  34]
 [145  95]]
